<a href="https://colab.research.google.com/github/rakaplay/cnn_recogn/blob/main/recogn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####**Это простая модель обучена для распознавания всего трех вещей:**
#### - Очков;
#### - Салфеток;
#### - Блока зарядки.

### Следующий код скачает и распакует датасет:

In [ ]:
import requests
response = requests.get("https://nekofile.eu.org/2cb5d44c214e47e68365azq0f")
file = open("data_and_model.tar", "wb")
file.write(response.content)
!tar -xf data_and_model.tar
file.close()
del response

### А при помощи кода ниже можно обучить модель:

In [ ]:
# библиотеки
import json
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [ ]:
# загрузка

# Чтение JSON файла
with open('data.json', 'r') as f:
    data = json.load(f)

# Параметры
img_size = (128, 128)
num_classes = len(data)

# Функция для загрузки изображений и меток
def load_data(data):
    images = []
    labels = []
    for class_id, items in data.items():
        for item in items:
            img_path, label = item
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(int(label))
    return np.array(images), np.array(labels)

# Загрузка данных
images, labels = load_data(data)
labels -= 1
# Нормализация изображений
images = images.astype('float32') / 255.0
print(images[0])
images = np.expand_dims(images, axis=-1)

# Преобразование меток в one-hot encoding
labels = to_categorical(labels, num_classes=num_classes)

# Разделение данных на тренировочные и валидационные наборы
from sklearn.model_selection import train_test_split
X_train, y_train = images, labels

[[0.83137256 0.8392157  0.83137256 ... 0.56078434 0.53333336 0.49019608]
 [0.8235294  0.8352941  0.80784315 ... 0.69803923 0.6313726  0.5764706 ]
 [0.7411765  0.7529412  0.7607843  ... 0.74509805 0.7372549  0.7490196 ]
 ...
 [0.53333336 0.54509807 0.5411765  ... 0.6        0.6156863  0.6156863 ]
 [0.54901963 0.54901963 0.5411765  ... 0.6039216  0.61960787 0.62352943]
 [0.5372549  0.54509807 0.54509807 ... 0.5058824  0.47843137 0.44705883]]


In [ ]:

# Создание модели и обучение
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", input_shape=(128, 128, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="gelu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
# Обучение модели
model.fit(X_train, y_train, epochs=30, batch_size=96)
model.save('recognizer.keras') # сохранение

In [ ]:
# Функция для предсказания класса объекта на изображении
def predict_class(image_path):
    # Чтение изображения
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    names = {0: 'ochki', 1: 'zaryadka', 2: 'salfetki'}
    # Изменение размера изображения
    img = cv2.resize(img, img_size)

    # Нормализация изображения
    img = img.astype('float32') / 255.0

    # Добавление оси для соответствия форме входных данных модели
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)

    # Предсказание
    prediction = model.predict(img)
    # Получение класса с наибольшей вероятностью
    class_id = np.argmax(prediction)
    preds = list(prediction.reshape(3))
    preds_dict = dict()
    for i in range(3):
        preds_dict[names[i]] = round(preds[i], 4)
    predicted_class_name = {0: 'ochki', 1: 'zaryadka', 2: 'salfetki'}[class_id]
    return predicted_class_name, preds_dict

In [ ]:
# Пример использования функции для предсказания класса.
# Если выдает zaryadka, то все правильно.
img_size = (128, 128)
image_path = "class3/3.png"
predicted_class = predict_class(image_path)
print(f'Предсказанный класс: {predicted_class[0]}.\nВероятности:\n{predicted_class[1]}')

1/1 [==============================] - 0s 25ms/step
Предсказанный класс: salfetki.
Вероятности:
{'ochki': 0.0, 'zaryadka': 0.0, 'salfetki': 1.0}


### Ниже предоставлен интерактивный режим на OpenCV. Правда, он не работает в Colab, нужно проверять на своем ПК.

In [ ]:

# Загрузка предобученной модели
model = tf.keras.models.load_model('pretrain.keras')

# Функция для предсказания класса объекта на изображении
def predict_class(image):
    # Изменение размера изображения
    img = cv2.resize(image, img_size)

    # Нормализация изображения
    img = img.astype('float32') / 255.0

    # Добавление оси для соответствия форме входных данных модели
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)

    # Предсказание
    prediction = model.predict(img)
    # Получение класса с наибольшей вероятностью
    class_id = np.argmax(prediction)

    return class_id

# Захват видео с камеры
cap = cv2.VideoCapture(0)

while True:
    # Чтение кадра
    ret, frame = cap.read()

    if not ret:
        break

    # Преобразование кадра в оттенки серого
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Предсказание класса
    predicted_class = predict_class(gray)
    predicted_class = {0: 'ochki', 1: 'zaryadka', 2: 'salfetki'}[predicted_class]
    # Отображение предсказанного класса на кадре
    cv2.putText(frame, f'This is {predicted_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Показ кадра
    cv2.imshow('Frame', frame)

    # Выход из цикла при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

### Ниже есть реализация для Google Colab:

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# Замените 'pretrain.keras' на путь к вашей предобученной модели
model = tf.keras.models.load_model('pretrain.keras')
img_size = (128, 128)

def predict_class(image):
    img = cv2.resize(image, img_size)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    class_id = np.argmax(prediction)
    return class_id
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.2):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.style.width = "5cm"
      capture.style.padding = "20px 0px";
      capture.style.borderRadius = "20px";
      capture.style.hover="{padding: 30px 0px}"
      capture.style.border = "1px solid #000"
      capture.textContent = 'Сделать фото';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.borderRadius = "20px";
      video.style.width = "5cm";
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename
filename = take_photo()
image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

# Предсказание класса
predicted_class = predict_class(image)
predicted_class = {0: 'очки', 1: 'зарядка', 2: 'салфетки'}[predicted_class]

# Отображение результата
print(f'This is {predicted_class}')

<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 94ms/step
This is зарядка
